In [31]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Literal, NotRequired
from pydantic import BaseModel, Field
from google import genai
import os
from google.genai.types import GenerateContentConfig

In [32]:
load_dotenv()
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

In [33]:
#TEST

# 1. Load the environment variables
loaded = load_dotenv()

# 2. Get the key
api_key = os.getenv("GOOGLE_API_KEY") # Make sure this name matches your .env file exactly

print(f"1. Did .env file load? -> {loaded}")
print(f"2. Is api_key variable found? -> {api_key is not None}")

if api_key:
    # Print only the first 4 characters for safety
    print(f"3. Key starts with: {api_key[:4]}...")
    print(f"4. Key length: {len(api_key)}")
else:
    print("3. RESULT: The code cannot read the variable. Check your .env file.")

1. Did .env file load? -> True
2. Is api_key variable found? -> True
3. Key starts with: AIza...
4. Key length: 39


In [34]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(description="The overall sentiment of the text.")

class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description="List of identified issues in the review.")
    tone: Literal["neutral", "angry", "frustrated", "sad", "disappointed", "calm"] = Field(description="The tone of the review.")
    urgency: Literal["low", "medium", "high"] = Field(description="The urgency and critical level of the review.")

In [35]:


if not api_key:
    raise ValueError("Error: GEMINI_API_KEY is missing from .env file.")

# 3. CRITICAL FIX: Initialize Client with the key here
# Do NOT rely on automatic detection if it is failing.
client = genai.Client(api_key=api_key)

schema = SentimentSchema.model_json_schema()

#parsing
config = GenerateContentConfig(
    response_mime_type="application/json",
    response_schema=schema
)

# Use "gemini-1.5-flash" or "gemini-2.0-flash-exp"
try:
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents="What is the sentiment of the following reviews - Software is too good.",
        config=config
    )
    
    # 5. Validate and print
    result = SentimentSchema.model_validate_json(response.text)
    print(f"Sentiment: {result.sentiment}")

except Exception as e:
    print(f"An error occurred: {e}")

Sentiment: positive


In [36]:
from typing import NotRequired

class ReviewState(TypedDict):
    review_text: str
    sentiment: NotRequired[Literal["positive", "negative"]]
    diagnosis: NotRequired[dict]
    response: NotRequired[str]

In [37]:
def find_sentiment(state: ReviewState) -> ReviewState:
    # 3. CRITICAL FIX: Initialize Client with the key here
    prompt = f"For the followoing review find out the sentiment \n {state['review_text'] }"
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=prompt,
        config=config
    )
    result = SentimentSchema.model_validate_json(response.text)
    return {'sentiment': result.sentiment}


In [ ]:
# Define the conditional workflow graph
def check_sentiment(state: ReviewState) -> Literal["positive_response", "run_diagnosis"]:
    sentiment = state.get('sentiment', '')
    if sentiment == 'positive':
        return 'positive_response'
    else:
        return 'run_diagnosis'


In [42]:
def positive_response(state: ReviewState) -> ReviewState:
    prompt = f"Write a warm thank you reply to the following positive review:\n\n {state['review_text']}. \n Also ask user to leave feedback on our website."
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=prompt
    )
    result = response.text
    return {'response': result}


def run_diagnosis(state: ReviewState) -> ReviewState:
    # Create config for DiagnosisSchema
    diagnosis_schema = DiagnosisSchema.model_json_schema()
    diagnosis_config = GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=diagnosis_schema
    )
    
    prompt = f"Diagnose the issues in the following negative review and suggest improvements:\n\n {state['review_text']}. \n Return issue_type, tone and urgency."
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=prompt,
        config=diagnosis_config
    )
    result = DiagnosisSchema.model_validate_json(response.text)
    return {'diagnosis': result.model_dump()}

def negative_response(state: ReviewState) -> ReviewState:
    diagnosis = state.get('diagnosis', {})
    prompt = f"Write a professional empathetic reply to the following negative review addressing {diagnosis.get('issue_type', 'the issues')} with a {diagnosis.get('tone', 'professional')} tone."
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=prompt
    )
    result = response.text
    return {'response': result}


In [43]:

graph = StateGraph(ReviewState)

graph.add_node('find_sentiment', find_sentiment)
graph.add_node('positive_response', positive_response)
graph.add_node('run_diagnosis', run_diagnosis)
graph.add_node('negative_response', negative_response)

graph.add_edge(START, 'find_sentiment')
graph.add_conditional_edges('find_sentiment', check_sentiment)
graph.add_edge('positive_response', END)
graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response', END)

workflow = graph.compile()

initial_state = {
    'review_text': "The software is pathetic and crashes all the time. Customer support was unhelpful and rude."
}

final_state = workflow.invoke(initial_state)
print(final_state)


{'review_text': 'The software is pathetic and crashes all the time. Customer support was unhelpful and rude.', 'sentiment': 'negative', 'diagnosis': {'issue_type': 'Bug', 'tone': 'angry', 'urgency': 'high'}, 'response': 'Dear [Customer\'s Name, if known, otherwise "Valued Customer" or "Reviewer"],\n\nThank you for taking the time to share your feedback. I am truly sorry to hear about the frustration and anger you\'re experiencing with the bug you\'ve encountered. We understand how incredibly upsetting it can be when our product doesn\'t perform as expected, and for that, please accept our sincerest apologies.\n\nThis is certainly not the experience we want any of our users to have, and we are very concerned to learn about this issue. We take bug reports seriously, and your detailed feedback is invaluable in helping us identify and fix problems to improve our service for everyone.\n\nOur development team has been alerted and is actively investigating reports like yours with the highest 